In [1]:
import sys
sys.path.append("/home/test/PycharmProjects")
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import pandas as pd
import json

In [2]:
annotation_file = './example/captions_val2014.json'
results_file = './example/captions_val2014_fakecap_results.json'

# create coco object and coco_result object
# coco = COCO(annotation_file)

In [3]:
# create coco_eval object by taking coco and coco_result
# coco_result = coco.loadRes(results_file)

In [4]:
! ls .

bleu					 pycocoevalcap.egg-info
cider					 README.md
df_av_test_after_fat.pk			 rouge
df_av_test_before_fat.pk		 setup.py
drive-download-20220901T162327Z-001.zip  spice
eval.py					 test_after_trained_on_all_ped.pk
example					 test_before_trained_on_all_ped.pk
license.txt				 tokenizer
meteor					 venv
metrics.ipynb				 wit_av_test_after.pk
metrics-ped.ipynb			 wit_ped_test_after.pk
__pycache__


In [5]:

with open(annotation_file, 'r') as f:
    jdata = json.load(f)

In [6]:
for el in jdata:
    print(el)

info
images
licenses
type
annotations


In [7]:
d_path = '.'
bf = f'{d_path}/df_av_test_before_fat.pk'
af_wit = f'{d_path}/wit_av_test_after.pk'
af_fat = f'{d_path}/df_av_test_after_fat.pk'

In [45]:
# df_wp = pd.read_pickle(bf)
# df_wp = pd.read_pickle(af_wit)
df_wp = pd.read_pickle(af_fat)
df_wp

,uniq_id,image_id,caption,topic_id,labels,image,topic2caption_sim,concept2caption_sim,topics,tag,ofa_caption,gt
0,4,4,Firefighting Drones,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.512088,0.489315,a,"{'aircraft': 0, 'drone': 0, 'sensor': 0, 'uav'...",Airmen with a drone in front of a gas pipeline,uav uavs unmanned uas
1,27,27,DJI mavic 2 pro in flight,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.506123,0.303162,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna A drone flying in the air,drone uav uavs unmanned uas
2,51,51,A small UAS is seen flying during at the US Ar...,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.456664,0.440849,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",A drone flies through the air with arrows in t...,drone uav uavs unmanned uas
3,102,102,Ruko F11 GIM2 Drone with Camera for Adults 4K ...,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.425304,0.273581,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna B17 with 4 Drones and 4 Brones with 4 d,drone uav uavs unmanned uas
4,130,130,The MQ9 Reaper,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.246585,0.226430,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna B17 B17 in flight,drone uav uavs unmanned uas
...,...,...,...,...,...,...,...,...,...,...,...,...
677,1931,1931,In our globalized world aviation increasingly ...,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.265668,0.481348,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",Airports logo on the door of an airplane,aircraft airline
678,1947,1947,Lufthansa Business Class on longhaul aircraft,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.333263,0.526307,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",A passenger sleeps in the aisle of an airplane,aircraft airline flight
679,1969,1969,The Boeing 7478 has inherited the good looks o...,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.309636,0.431361,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",Lufthansa is the worlds largest airline,aircraft airline plane flight
680,1979,1979,911 Attacks Pentagon,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.655778,0.354867,j,"{'aircraft': 1, 'airline': 0, 'tower': 0, '911...",A painting of two pilots in a cockpit with a f...,aircraft pentagon plane flight


In [46]:
def create_gt(tag):
    context = []
    for el in tag:
        if tag[el]:
            context.append(el)
    if not context:
        context = None
    return context

In [47]:
def create_con_gt(tag):
    context = ""
    for el in tag:
        if tag[el]:
            context = context + " " + el
    if not context:
        context = None
    return context

In [48]:
df_wp['gt'] = df_wp.apply(lambda x: create_gt(x.tag), axis=1)
df_wp['con_gt'] = df_wp.apply(lambda x: create_con_gt(x.tag), axis=1)
df_wp

,uniq_id,image_id,caption,topic_id,labels,image,topic2caption_sim,concept2caption_sim,topics,tag,ofa_caption,gt,con_gt
0,4,4,Firefighting Drones,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.512088,0.489315,a,"{'aircraft': 0, 'drone': 0, 'sensor': 0, 'uav'...",Airmen with a drone in front of a gas pipeline,"[uav, uavs, unmanned, uas]",uav uavs unmanned uas
1,27,27,DJI mavic 2 pro in flight,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.506123,0.303162,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna A drone flying in the air,"[drone, uav, uavs, unmanned, uas]",drone uav uavs unmanned uas
2,51,51,A small UAS is seen flying during at the US Ar...,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.456664,0.440849,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",A drone flies through the air with arrows in t...,"[drone, uav, uavs, unmanned, uas]",drone uav uavs unmanned uas
3,102,102,Ruko F11 GIM2 Drone with Camera for Adults 4K ...,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.425304,0.273581,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna B17 with 4 Drones and 4 Brones with 4 d,"[drone, uav, uavs, unmanned, uas]",drone uav uavs unmanned uas
4,130,130,The MQ9 Reaper,31,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.246585,0.226430,a,"{'aircraft': 0, 'drone': 1, 'sensor': 0, 'uav'...",Cessna B17 B17 in flight,"[drone, uav, uavs, unmanned, uas]",drone uav uavs unmanned uas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,1931,1931,In our globalized world aviation increasingly ...,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.265668,0.481348,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",Airports logo on the door of an airplane,"[aircraft, airline]",aircraft airline
678,1947,1947,Lufthansa Business Class on longhaul aircraft,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.333263,0.526307,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",A passenger sleeps in the aisle of an airplane,"[aircraft, airline, flight]",aircraft airline flight
679,1969,1969,The Boeing 7478 has inherited the good looks o...,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.309636,0.431361,j,"{'aircraft': 1, 'airline': 1, 'tower': 0, '911...",Lufthansa is the worlds largest airline,"[aircraft, airline, plane, flight]",aircraft airline plane flight
680,1979,1979,911 Attacks Pentagon,220,,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,0.655778,0.354867,j,"{'aircraft': 1, 'airline': 0, 'tower': 0, '911...",A painting of two pilots in a cockpit with a f...,"[aircraft, pentagon, plane, flight]",aircraft pentagon plane flight


In [49]:
print(df_wp['gt'].isna().sum())
df_wp = df_wp[df_wp['gt'].notna()]
df_wp = df_wp.reset_index(drop=True)
len(df_wp)

0


682

In [50]:
import numpy as np

In [51]:
def id2imdf(_df):
    _df = _df[['uniq_id']]
    _df['license'] = 1
    _df['url'] = 'http://farm9.staticflickr.com'
    _df['file_name'] = 'test.jpg'
    _df['width'] = 640
    _df['height'] = 360
    _df['date_captured'] = '2013-11-14 11:18:45'
    _df.columns = _df.columns.str.replace('uniq_id', 'id')
    return _df

In [52]:
def concat_gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
        caption.append(row['con_gt'])
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [53]:
def gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
#         print(row['image_id'], row['gt'])
        for el in row['gt']:
            image_id.append(row['image_id'])
            caption.append(el)
#         break
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [54]:
def org_2anno(_df):
    image_id = []
#     caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
#         caption.append('pedestrian')
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(image_id))
    df_['caption'] = _df['caption'].to_list()
    return df_

In [55]:
gk = df_wp.groupby('topics')
pg_dict = dict()
for el in gk.groups.keys():
    pg_dict[el] = dict()
    pg_dict[el]['data'] = gk.get_group(el)
    pg_dict[el]['data'] = pg_dict[el]['data'][pg_dict[el]['data']['topic2caption_sim'] >= pg_dict[el]['data']['topic2caption_sim'].quantile(.25)]
    pg_dict[el]['data'] = pg_dict[el]['data'].reset_index(drop=True)
    temp = pg_dict[el]['data'][['image_id', 'ofa_caption']]
    temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
    pg_dict[el]['results'] = temp
    pg_dict[el]['images'] = id2imdf(pg_dict[el]['data'])
    pg_dict[el]['split_annotations'] = gt2anno(pg_dict[el]['data'])
    pg_dict[el]['con_annotations'] = concat_gt2anno(pg_dict[el]['data'])
    pg_dict[el]['org_annotations'] = org_2anno(pg_dict[el]['data'])

/tmp/ipykernel_7796/2853690035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['license'] = 1
/tmp/ipykernel_7796/2853690035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['url'] = 'http://farm9.staticflickr.com'
/tmp/ipykernel_7796/2853690035.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [56]:
pg_dict[el]['org_annotations']

,image_id,id,caption
0,499,0,One of many control panels in aircraft simulat...
1,507,1,Cartoon scene with military army car vehicle o...
2,508,2,Cockpit B1900 stock photos
3,518,3,Many switches in an aircraft cockpit royalty f...
4,520,4,View of an old aircraft cockpit with two yokes...
...,...,...,...
76,1792,76,The various air traffic control facilities enc...
77,1795,77,Its Our Passion Airspace Aviation
78,1947,78,Lufthansa Business Class on longhaul aircraft
79,1979,79,911 Attacks Pentagon


In [57]:
def crate_json(t='split_annotations'):
    t_data = dict()
    t_data['info'] = jdata['info']
    t_data['images'] = pg_dict[el]['images'].to_dict('records')
    t_data['type'] = jdata['type']
    t_data['licenses'] = jdata['licenses']
    t_data['annotations'] = pg_dict[el][t].to_dict('records')
    return t_data
    

In [58]:
file_dict = dict()
for el in gk.groups.keys():
    file_dict[el] = dict()
#     print(el)
    
    s_data = crate_json(t='split_annotations')
    c_data = crate_json(t='con_annotations')  
    o_data = crate_json(t='org_annotations')  
    res = pg_dict[el]['results'].to_dict('records')
    
    file_dict[el]['results'] = f"data/{el}_results.json"
    with open(file_dict[el]['results'], "w") as final:
        json.dump(res, final)
    
    file_dict[el]['sval'] = f"data/{el}_sval.json"
    with open(file_dict[el]['sval'], "w") as final:
        json.dump(s_data, final)
    
    file_dict[el]['cval'] = f"data/{el}_cval.json"
    with open(file_dict[el]['cval'], "w") as final:
        json.dump(c_data, final)
    
    file_dict[el]['oval'] = f"data/{el}_oval.json"
    with open(file_dict[el]['oval'], "w") as final:
        json.dump(o_data, final)

# Load data

In [59]:
def calc_metrics(topic='a', anno='s'):
    if anno == 's':
        annotation_file = file_dict[topic]['sval']
    elif anno == 'c':
        annotation_file = file_dict[topic]['cval']
    else:
        annotation_file = file_dict[topic]['oval']
        
    results_file = file_dict[topic]['results']
    coco = COCO(annotation_file)
    coco_result = coco.loadRes(results_file)
    coco_eval = COCOEvalCap(coco, coco_result)
    coco_eval.params['image_id'] = coco_result.getImgIds()
    coco_eval.evaluate()
    return coco_eval.eval


In [60]:
split_res_dict = dict()
con_res_dict = dict()
org_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    org_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 102 tokens at 2360.94 tokens per second.
PTBTokenizer tokenized 97 tokens at 2224.75 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 88, 'reflen': 93, 'guess': [88, 78, 68, 58], 'correct': [7, 0, 0, 0]}
ratio: 0.9462365591296104
Bleu_1: 0.075
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.028
computing Rouge score...
ROUGE_L: 0.067
computing CIDEr score...
CIDEr: 0.073
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.6 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4

SPICE evaluation took: 3.579 s
SPICE: 0.159
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


...

SPICE evaluation took: 7.739 s
SPICE: 0.098
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 46 tokens at 542.33 tokens per second.
PTBTokenizer tokenized 52 tokens at 1096.52 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 48, 'reflen': 42, 'guess': [48, 43, 38, 33], 'correct': [2, 0, 0, 0]}
ratio: 1.1428571428299321
Bleu_1: 0.042
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.007
computing Rouge score...
ROUGE_L: 0.051
computing CIDEr score...
CIDEr: 0.117
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.7 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4 sec].
Loading classif

SPICE evaluation took: 6.013 s
SPICE: 0.000


# Av before

In [25]:
all_res = []
for el in gk.groups.keys():
    all_res.append(org_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(4)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0787,0.0297,0.0090,0.0027,0.0347,0.0809,0.0922,0.0459
std,0.0359,0.0170,0.0112,0.0052,0.0130,0.0333,0.0642,0.0352
min,0.0219,0.0091,0.0000,0.0000,0.0187,0.0345,0.0305,0.0169
25%,0.0546,0.0141,0.0000,0.0000,0.0260,0.0566,0.0404,0.0221
50%,0.0784,0.0315,0.0042,0.0000,0.0316,0.0754,0.0614,0.0301
75%,0.1022,0.0385,0.0128,0.0020,0.0435,0.1033,0.1391,0.0636
max,0.1363,0.0604,0.0282,0.0156,0.0599,0.1312,0.2077,0.1272


In [26]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in org_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.118
Bleu_2: 0.038
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.046
ROUGE_L: 0.087
CIDEr: 0.037
SPICE: 0.127

b
Bleu_1: 0.098
Bleu_2: 0.036
Bleu_3: 0.013
Bleu_4: 0.000
METEOR: 0.048
ROUGE_L: 0.131
CIDEr: 0.208
SPICE: 0.075

c
Bleu_1: 0.054
Bleu_2: 0.010
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.026
ROUGE_L: 0.058
CIDEr: 0.064
SPICE: 0.033

d
Bleu_1: 0.057
Bleu_2: 0.021
Bleu_3: 0.012
Bleu_4: 0.008
METEOR: 0.023
ROUGE_L: 0.034
CIDEr: 0.031
SPICE: 0.017

e
Bleu_1: 0.022
Bleu_2: 0.009
Bleu_3: 0.005
Bleu_4: 0.003
METEOR: 0.019
ROUGE_L: 0.052
CIDEr: 0.035
SPICE: 0.027

f
Bleu_1: 0.082
Bleu_2: 0.027
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.056
CIDEr: 0.059
SPICE: 0.019

g
Bleu_1: 0.075
Bleu_2: 0.046
Bleu_3: 0.028
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.079
CIDEr: 0.170
SPICE: 0.027

h
Bleu_1: 0.041
Bleu_2: 0.012
Bleu_3: 0.003
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.072
CIDEr: 0.049
SPICE: 0.043

i
Bleu_1: 0.104
Bleu_2: 0.039
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.109
C

# Av after wit

In [43]:
# org_res_dict
all_res = []
for el in gk.groups.keys():
    all_res.append(org_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(4)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0953,0.0432,0.0174,0.0077,0.0421,0.1054,0.1372,0.0598
std,0.0383,0.0226,0.0177,0.0122,0.0119,0.0366,0.0957,0.0363
min,0.0358,0.0138,0.0000,0.0000,0.0299,0.0530,0.0524,0.0222
25%,0.0814,0.0240,0.0062,0.0000,0.0338,0.0824,0.0622,0.0354
50%,0.0934,0.0445,0.0122,0.0000,0.0396,0.1002,0.1112,0.0453
75%,0.1174,0.0561,0.0238,0.0129,0.0443,0.1197,0.1922,0.0935
max,0.1603,0.0797,0.0512,0.0302,0.0669,0.1623,0.3436,0.1158


In [44]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in org_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.080
Bleu_2: 0.028
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.081
CIDEr: 0.062
SPICE: 0.109

b
Bleu_1: 0.118
Bleu_2: 0.051
Bleu_3: 0.016
Bleu_4: 0.000
METEOR: 0.057
ROUGE_L: 0.162
CIDEr: 0.195
SPICE: 0.107

c
Bleu_1: 0.092
Bleu_2: 0.023
Bleu_3: 0.010
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.085
CIDEr: 0.063
SPICE: 0.047

d
Bleu_1: 0.085
Bleu_2: 0.039
Bleu_3: 0.014
Bleu_4: 0.000
METEOR: 0.030
ROUGE_L: 0.053
CIDEr: 0.054
SPICE: 0.022

e
Bleu_1: 0.041
Bleu_2: 0.018
Bleu_3: 0.007
Bleu_4: 0.000
METEOR: 0.032
ROUGE_L: 0.070
CIDEr: 0.065
SPICE: 0.035

f
Bleu_1: 0.117
Bleu_2: 0.052
Bleu_3: 0.026
Bleu_4: 0.016
METEOR: 0.044
ROUGE_L: 0.119
CIDEr: 0.158
SPICE: 0.036

g
Bleu_1: 0.095
Bleu_2: 0.071
Bleu_3: 0.051
Bleu_4: 0.030
METEOR: 0.044
ROUGE_L: 0.108
CIDEr: 0.344
SPICE: 0.044

h
Bleu_1: 0.036
Bleu_2: 0.014
Bleu_3: 0.006
Bleu_4: 0.003
METEOR: 0.033
ROUGE_L: 0.093
CIDEr: 0.052
SPICE: 0.053

i
Bleu_1: 0.130
Bleu_2: 0.057
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.044
ROUGE_L: 0.120
C

# Av after fat

In [61]:
all_res = []
for el in gk.groups.keys():
    all_res.append(org_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(4)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,0.0936,0.0476,0.0314,0.0192,0.0497,0.1282,0.2578,0.0842
std,0.0523,0.0383,0.0288,0.0249,0.0295,0.0616,0.1624,0.0543
min,0.0295,0.0000,0.0000,0.0000,0.0071,0.0510,0.0550,0.0000
25%,0.0747,0.0199,0.0061,0.0000,0.0303,0.0812,0.1304,0.0467
50%,0.0798,0.0482,0.0297,0.0089,0.0459,0.1272,0.2594,0.0742
75%,0.0975,0.0569,0.0452,0.0325,0.0607,0.1614,0.3366,0.1172
max,0.1890,0.1204,0.0916,0.0731,0.1106,0.2560,0.5555,0.1709


In [62]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in org_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.075
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.067
CIDEr: 0.073
SPICE: 0.159

b
Bleu_1: 0.180
Bleu_2: 0.120
Bleu_3: 0.092
Bleu_4: 0.073
METEOR: 0.111
ROUGE_L: 0.256
CIDEr: 0.440
SPICE: 0.171

c
Bleu_1: 0.085
Bleu_2: 0.058
Bleu_3: 0.048
Bleu_4: 0.042
METEOR: 0.046
ROUGE_L: 0.124
CIDEr: 0.555
SPICE: 0.124

d
Bleu_1: 0.080
Bleu_2: 0.047
Bleu_3: 0.025
Bleu_4: 0.000
METEOR: 0.037
ROUGE_L: 0.096
CIDEr: 0.297
SPICE: 0.062

e
Bleu_1: 0.030
Bleu_2: 0.012
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.076
CIDEr: 0.055
SPICE: 0.038

f
Bleu_1: 0.102
Bleu_2: 0.049
Bleu_3: 0.033
Bleu_4: 0.024
METEOR: 0.050
ROUGE_L: 0.131
CIDEr: 0.350
SPICE: 0.073

g
Bleu_1: 0.074
Bleu_2: 0.053
Bleu_3: 0.038
Bleu_4: 0.000
METEOR: 0.045
ROUGE_L: 0.131
CIDEr: 0.222
SPICE: 0.042

h
Bleu_1: 0.080
Bleu_2: 0.044
Bleu_3: 0.027
Bleu_4: 0.018
METEOR: 0.064
ROUGE_L: 0.172
CIDEr: 0.172
SPICE: 0.075

i
Bleu_1: 0.189
Bleu_2: 0.092
Bleu_3: 0.053
Bleu_4: 0.035
METEOR: 0.080
ROUGE_L: 0.178
C